# Importing necessary libraries and modules

In [1]:
from tkinter import*
from tkinter import ttk
from PIL import ImageTk,Image
from googletrans import Translator, LANGUAGES
from langdetect import detect
import speech_recognition as sr
import googletrans
import gtts
import playsound
import time
from tkinter import messagebox

# Function to handle the event when the input text entry is clicked

In [2]:
def on_entry_click(event):
    if input_text_entry.get() == "Enter text":
        input_text_entry.delete(0, "end")
        input_text_entry.configure(fg='black')

# Function to handle the event when the input text entry loses focus

In [3]:
def on_focus_out(event):
    if input_text_entry.get() == "":
        input_text_entry.insert(0, "Enter text")
        input_text_entry.configure(fg='gray')

# Function to detect the language of the input text

In [4]:
def detect_language():
    text = input_text_entry.get()
    if text != "Enter text":
        detected_lang = detect(text)
        return detected_lang
    return None

# Function to perform translation

In [5]:
def translate():
    text = input_text_entry.get()
    src_language = src_lang.get()
    dest_language = dest_lang.get()

    if text != "Enter text" and src_language != 'choose language' and dest_language != 'choose language':
        translator = Translator()
        
        # If the source language is set to 'Detect', try to detect the language automatically
        if src_language == 'Detect':
            detected_lang = detect_language()
            src_language = detected_lang if detected_lang else 'choose language'
        
        # Translate the text
        translated = translator.translate(text=text, src=src_language, dest=dest_language)
        
        # Update the output text area with the translated text
        output_text.delete("1.0", "end")
        output_text.insert("1.0", translated.text)
        
        # Show a message box indicating the translation is complete
        messagebox.showinfo("Translation Complete", "Translation is done!")

# Function to perform voice input and convert it to text

In [6]:
def voice_input():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        # Use the microphone as the audio source
        audio = recognizer.listen(source)
        try:
            src_language = src_lang.get()
            
            # If the source language is set to a specific language, convert it to the corresponding language code
            if src_language != 'choose language' and src_language != 'Detect':
                language_codes = {v: k for k, v in LANGUAGES.items()}
                src_language_code = language_codes.get(src_language, 'choose language')
                
                # Use the Google Speech Recognition API to recognize the speech and convert it to text
                recognized_text = recognizer.recognize_google(audio, language=src_language_code)
                input_text_entry.delete(0, "end")
                input_text_entry.insert(0, recognized_text)
            else:
                recognized_text = recognizer.recognize_google(audio)
                input_text_entry.delete(0, "end")
                input_text_entry.insert(0, recognized_text)
        except sr.UnknownValueError:
            messagebox.showinfo("Error","Speech recognition could not understand audio")
        except sr.RequestError as e:
            messagebox.showinfo("Could not request results from the speech recognition service; {0}".format(e))

# Function to generate voice output of the translated text

In [7]:
def voice_output():
    dest_language = dest_lang.get()
    
    # Function to generate voice output of the translated text
    language_codes = {v: k for k, v in LANGUAGES.items()}
    dest_language_code = language_codes.get(dest_language, 'choose language')
    
    # Get the translated text from the output text area
    translated_text = output_text.get("1.0", "end")
    
    # Generate a unique timestamp for the audio filename
    timestamp = str(int(time.time()))  
    
    # Create the filename for the audio file
    audio_filename = "translated_audio_" + timestamp + ".mp3" 
    
    # Use gtts library to convert the translated text to speech audio
    converted_audio = gtts.gTTS(translated_text, lang=dest_language_code)
    
    # Save the audio file
    converted_audio.save(audio_filename)
    
    # Play the audio file
    playsound.playsound(audio_filename)

# Creating the main root window

In [8]:
root = Tk()
root.title('Translator App')
root.geometry('1000x500')
root.state('zoomed')
root.resizable(0, 0)
root.configure(bg='#736a86')

frame = Frame(root, width=1245, height=650, bg='#edf1fe')
frame.place(x=150, y=100)

frame = Frame(root, width=575, height=650, bg='#b73275')
frame.place(x=820, y=100)

heading = Label(root, text='SMART  TRANSLATE', font=('forte', 30, 'bold'), bg='#edf1fe', fg='#464196')
heading.place(x=270, y=250)

side_image = Image.open('Image\\bg.png')
side_image = side_image.resize((571, 471)) 
photo = ImageTk.PhotoImage(side_image)
side_image_label = Label(root, image=photo, bg='#b73275')
side_image_label.image = photo
side_image_label.place(x=820, y=170)

input_text_label = Label(root, text='From', bg='#edf1fe', fg='#739bd0', font=('Yu Gothic UI bold', 18, 'bold'))
input_text_label.place(x=190, y=336)

input_text_entry = Entry(root, highlightthickness=0, relief="flat", width=30, bg='#dae4ee', fg='gray',
                         font=('Yu Gothic UI Semibold', 13))
input_text_entry.insert(0, "Enter text")
input_text_entry.bind("<FocusIn>", on_entry_click)
input_text_entry.bind("<FocusOut>", on_focus_out)
input_text_entry.place(x=190, y=400)

output_text_label = Label(root, text='To', bg='#edf1fe', fg='#739bd0', font=('Yu Gothic UI bold', 18, 'bold'))
output_text_label.place(x=505, y=336)

output_text = Text(root, highlightthickness=0, relief="flat", width=30, height=5, bg='#dae4ee', fg='black',
                   font=('Yu Gothic UI Semibold', 13), wrap='word')
output_text.place(x=505, y=400)

language1 = ['choose language', 'Detect'] + list(LANGUAGES.values())
src_lang = ttk.Combobox(root, values=language1, width=22)
src_lang.place(x=260, y=346)
src_lang.set('choose language')

language2 = ['choose language'] + list(LANGUAGES.values())
dest_lang = ttk.Combobox(root, values=language2, width=22)
dest_lang.place(x=550, y=346)
dest_lang.set('choose language')

trans_button = Image.open('Image\\trans.png')
resized_trans_image = trans_button.resize((200, 100))
photo = ImageTk.PhotoImage(resized_trans_image)
trans_button_label = Button(root, image=photo, bg='#edf1fe', activebackground="#edf1fe", cursor='hand2', bd=0, command=translate)
trans_button_label.image=photo
trans_button_label.place(x=358, y=527)

voice_input_button = Image.open('Image\\mic.png') 
resized_voice_image = voice_input_button.resize((30, 30))
photo = ImageTk.PhotoImage(resized_voice_image)
voice_input_button_label = Button(root, image=photo, bg='#edf1fe', activebackground="#edf1fe", cursor='hand2', bd=0, command=voice_input)
voice_input_button_label.image=photo
voice_input_button_label.place(x=420, y=340)

voice_output_button = Image.open('Image\\on.png') 
resized_image = voice_output_button.resize((30, 30))
photo = ImageTk.PhotoImage(resized_image)
voice_output_button_label = Button(root, image=photo, bg='#edf1fe', activebackground="#edf1fe", cursor='hand2', bd=0, command=voice_output)
voice_output_button_label.image=photo
voice_output_button_label.place(x=715, y=340)

root.mainloop()
